In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

import cv2
import os
import pydicom
import matplotlib.pyplot as plt

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
imlist = []

for dirname, _, filenames in os.walk('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000'):
    for filename in filenames:
        imlist.append(os.path.join(dirname, filename))

In [ ]:
def convert_from_dicom_to_png(img,filename):
    
    if np.max(img) == 0:
        
        new = img
    
    else:
        new = img/np.max(img)*255
    

    cv2.imwrite(filename,new) 

In [ ]:
mpMRI = ['FLAIR','T1w', 'T2w','T1wCE']

inputdir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/'
outdir = './'
#os.mkdir(outdir)

#test_list = [ f for f in  os.listdir(inputdir)]

#test_list[0]

dirs = [inputdir + mp + '/' for mp in mpMRI ]

In [ ]:
#plt.subplots(nrows = 10, ncols = 2, figsize=(50,50))

for directory in dirs:
    
    test_list = [ f for f in  os.listdir(directory)]
    
    #test_list

    for f in test_list[:3]:   # remove "[:10]" to convert all images 
        ds = pydicom.read_file(directory + f) # read dicom image
        img = ds.pixel_array # get image array
        print(ds)
        print('---------------------')
        print('-----DIRECTORY------')
        print(directory)
        print('---------------------')
        print('-----FILE NAME-------')
        print(inputdir + f)
        #cv2.imwrite(outdir + f.replace('.dcm','.png'),img) # write png image
        convert_from_dicom_to_png(img,outdir + f.replace('.dcm','.png'))
        plt.imshow(img,cmap=plt.cm.gray)
        plt.show()

## Playing around with pydicom

Playing round with the data I hope to find some solution to normalize the differnt Planes.

See here:

https://www.kaggle.com/davidbroberts/determining-mr-image-planes

In [ ]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob

# load the DICOM files
# Code was adapted from here https://pydicom.github.io/pydicom/stable/auto_examples/image_processing/reslice.html

inputdir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00009/T1w'

files = [ pydicom.dcmread(inputdir + '/' + f) for f in  os.listdir(inputdir)]

#test_list

print("file count: {}".format(len(test_list)))

# skip files with no SliceLocation (eg scout views)
slices = []
skipcount = 0
for f in files:
    if hasattr(f, 'SliceLocation'):
        slices.append(f)
    else:
        skipcount = skipcount + 1

print("skipped, no SliceLocation: {}".format(skipcount))

# ensure they are in the correct order
slices = sorted(slices, key=lambda s: s.SliceLocation)

# pixel aspects, assuming all slices are the same
ps = slices[0].PixelSpacing
ss = slices[0].SliceThickness
ax_aspect = ps[1]/ps[0]
sag_aspect = ps[1]/ss
cor_aspect = ss/ps[0]

# create 3D array
img_shape = list(slices[0].pixel_array.shape)
img_shape.append(len(slices))
img3d = np.zeros(img_shape)

# fill 3D array with the images from the files
for i, s in enumerate(slices):
    img2d = s.pixel_array
    img3d[:, :, i] = img2d

# plot 3 orthogonal slices
a1 = plt.subplot(2, 2, 1)
plt.imshow(img3d[:, :, img_shape[2]//2])
a1.set_aspect(ax_aspect)

a2 = plt.subplot(2, 2, 2)
plt.imshow(img3d[:, img_shape[1]//2, :])
a2.set_aspect(sag_aspect)

a3 = plt.subplot(2, 2, 3)
plt.imshow(img3d[img_shape[0]//2, :, :].T)
a3.set_aspect(cor_aspect)

plt.show()